In [247]:
import re
import plotly.express as px
import pandas as pd
import statsmodels
# pd.set_option("display.max_rows", None, "display.max_columns", None)

def added_weekly(fig, idx):
    results = px.get_trendline_results(fig)
    return results.iloc[idx]["px_fit_results"].params[1] * pd.to_timedelta(1, unit='d').total_seconds() * 7

def estimate(fig, idx, date=pd.Timestamp.now()):
    results = px.get_trendline_results(fig)
    return results.iloc[idx]["px_fit_results"].params[0] + results.iloc[idx]["px_fit_results"].params[1] * date.timestamp()

In [210]:
df = pd.read_csv('../output-sp2-2023-05-08/solve_other_d4.sh/output.csv')
df['committer_date'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
df = df.drop(columns=['kconfig-file', 'environment', 'binding-file', 'model-file', 'dimacs-file', 'dimacs-transformer', 'dimacs-time', 'model-time', 'smt-file', 'smt-time', 'smt-transformer', 'dimacs-analyzer', 'system', 'name', 'committer_date_readable', 'committer_date_unix'])
df['model-count-log10'] = (df['model-count']).fillna(0).map(lambda s: len(str(s))) #todo
df

,dimacs-literals,dimacs-variables,extractor,model-features,model-literals,model-variables,revision,source_lines_of_code,dimacs-analyzer-time,model-count,committer_date,model-count-log10
0,139320,4964,kconfigreader,3263,66561,3629,v2.5.45,3495214,2.451609e+10,4333176923880034954264649751641420249418464843...,2002-10-31 00:51:00,630
1,138845,4988,kconfigreader,3279,66794,3646,v2.5.46,3545394,5.999150e+10,1925860886421133126449898050094294361522394905...,2002-11-04 22:51:00,633
2,141442,5048,kconfigreader,3318,67376,3681,v2.5.47,3559012,2.181154e+10,6089637296520251191353663070482929071954867786...,2002-11-11 03:33:00,639
3,136721,5027,kconfigreader,3325,67076,3688,v2.5.48,3561154,3.633550e+10,1309416634037342397820276763400386239218913858...,2002-11-18 04:37:00,641
4,137243,5034,kconfigreader,3328,67092,3691,v2.5.49,3562374,3.261390e+10,6404793435923797083579971373510214107351732492...,2002-11-22 21:50:00,641
...,...,...,...,...,...,...,...,...,...,...,...,...
267,728306,39207,kmax,15774,228708,15151,v5.9,20493802,NaN,NaN,2020-10-11 21:15:50,1
268,823574,43667,kmax,17135,259424,16528,v6.0,24874347,NaN,NaN,2022-10-02 21:09:07,1
269,843747,44122,kmax,17304,262826,16697,v6.1,25190977,NaN,NaN,2022-12-11 22:15:18,1
270,865867,44503,kmax,17441,266150,16829,v6.2,25410670,NaN,NaN,2023-02-19 22:24:22,1


In [227]:
fig = px.scatter(df, x='committer_date', y='source_lines_of_code', hover_data=['revision'], trendline='ols')
print("~#SLOC added per week: " + str(int(added_weekly(fig, 0))))
fig

~#SLOC added per week: 18660


In [228]:
fig = px.scatter(df, x='committer_date', y='model-features', hover_data=['revision'], trendline='ols', color='extractor')
print("~#features added per week (kconfigreader): " + str(int(added_weekly(fig, 0))))
print("~#features added per week (kmax): " + str(int(added_weekly(fig, 1))))
fig

~#features added per week (kconfigreader): 22
~#features added per week (kmax): 14


In [229]:
fig = px.scatter(df, x='committer_date', y='dimacs-variables', hover_data=['revision'], trendline='ols', color='extractor')
print("~#variables added per week (kconfigreader): " + str(int(added_weekly(fig, 0))))
print("~#variables added per week (kmax): " + str(int(added_weekly(fig, 1))))
fig

~#variables added per week (kconfigreader): 54
~#variables added per week (kmax): 40


In [253]:
df2 = df[df['model-count-log10'] > 1]
fig = px.scatter(df2, x='committer_date', y='model-count-log10', hover_data=['revision'], color='extractor', trendline='ols')
print("~#configurations scaled per week (kconfigreader): " + str(int(pow(10, added_weekly(fig, 0)))))
print("~#configurations scaled per week (kmax): " + str(int(pow(10, added_weekly(fig, 1)))))
print("~#configurations today (kconfigreader): 10^" + str(int(estimate(fig, 0))))
print("~#configurations today (kmax): 10^" + str(int(estimate(fig, 1))))
fig

~#configurations scaled per week (kconfigreader): 37
~#configurations scaled per week (kmax): 12
~#configurations today (kconfigreader): 10^2322
~#configurations today (kmax): 10^1593


In [257]:
archdf = pd.read_csv('../output-sp2-2023-05-08/arch.csv')
archdf['committer_date_unix'] = archdf['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
fig = px.line(dfa, x='committer_date_unix', y='architecture', hover_data=['revision'], color='architecture')
fig.update_layout(height=1000)
fig

In [194]:
df = pd.read_csv('../output/kconfig/output.csv')
df['committer_date_unix'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))

fig = px.scatter(df, x='model-features', y='revision-parameter', color='extractor', size='model-literals')
fig.show()
#todo: for history, consider boxplot

In [198]:
fig = px.box(df, x='committer_date_unix', y='model-features', color='extractor')
fig.show()
#todo: for history, consider boxplot